In [1]:
`project_id setenv "ferenc-world"
`csbucketname setenv "storagebodon"

In [2]:
project: getenv `project_id
csbucketname: getenv `csbucketname

# From BigQuery to kdb

In [3]:
// extract from BigQuery to Cloud Storage
system "bq extract --destination_format NEWLINE_DELIMITED_JSON ", project, ":bqkdb.safeBQTypesArray gs://", csbucketname, "/safeBQTypesArray.json"

Waiting on bqjob_r4b332cc2600fcf7f_0000016e5c3871ae_1 ... (0s) Current status: DONE   

""


In [4]:
csfilename: "gs://", csbucketname, "/safeBQTypesArray.json"

In [5]:
// Copy from Cloud Storage to local box
system "gsutil cp ", csfilename, " /tmp/"

Copying gs://storagebodon/safeBQTypesArray.json...
- [1 files][  162.0 B/  162.0 B]                                                
Operation completed over 1 objects/162.0 B.                                      


In [6]:
\l utils/csvutil.q
\c 25 120

## Manual type conversion
* string, boolean and float types are properly handled
* other types need to be casted manually
* timestamps needs minor conversion, chopping timezone information off


In [7]:
decode: {[fM; j] 
  k: .j.k j; 
  key[k]!fM[key k]@'value k}

In [8]:
formatterMap: (enlist `)!enlist (::)   // default formatter: leave unchanged

In [9]:
formatterMap[`intArray]: "I"$

In [10]:
t: formatterMap decode/: read0 hsym `$"/tmp/safeBQTypesArray.json"

In [11]:
t

stringArray   floatArray  intArray
----------------------------------
"BGT"  "MA"   98.3  110.2 4  1000 
"GOOG" "AAPL" 100.3 104.9 42 200  


In [12]:
meta t

c          | t f a
-----------| -----
stringArray|      
floatArray | F    
intArray   | I    


# From kdb to BigQuery

Let us save the fixed kdb table to JSON

In [13]:
save `:/tmp/t.json

`:/tmp/t.json


In [14]:
system "bq load --autodetect --source_format NEWLINE_DELIMITED_JSON bqkdb.safeBQTypesArray_json /tmp/t.json"

Waiting on bqjob_r1600d59df943eae3_0000016e5c388668_1 ... (1s) Current status: DONE   

""
""


We can see that the **column orders do not match**

In [15]:
system "bq show bqkdb.safeBQTypesArray_json"

"Table ferenc-world:bqkdb.safeBQTypesArray_json"
""
"   Last modified                 Schema                 Total Rows   Total Bytes   Expiration   Time Partitioning   ..
" ----------------- ----------------------------------- ------------ ------------- ------------ ------------------- -..
"  11 Nov 21:47:33   |- intArray: integer (repeated)     4            170                                            ..
"                    |- floatArray: float (repeated)                                                                 ..
"                    |- stringArray: string (repeated)                                                               ..
""


## Cleanup

In [16]:
// Cloud Storage
system "gsutil rm ", csfilename

Removing gs://storagebodon/safeBQTypesArray.json...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              


In [17]:
// local files
system "rm /tmp/t.json"

In [18]:
// BigQuery table
system "bq rm -f bqkdb.allBQSimpleTypes_json"